In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
cd ..

/home/wbaik/Code/data_science/playground_pytorch


In [3]:
from postgres_utils import PostgresPandas

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
ppd = PostgresPandas()

In [5]:
queries = '''
select *
from spx_table
order by "Sector";
'''

In [6]:
ret = ppd.run_query(queries)

In [7]:
ret.head()

,index,Symbol,Name,Sector
0,465,URBN,Urban Outfitters,Consumer Discretionary
1,229,HD,Home Depot,Consumer Discretionary
2,286,M,Macy's Inc.,Consumer Discretionary
3,282,LOW,Lowe's Cos.,Consumer Discretionary
4,456,UAA,Under Armour,Consumer Discretionary


In [8]:
second_query = '''
select * from aapl
limit 100'''

In [9]:
aapl_100 = ppd.run_query(second_query)

In [10]:
aapl_100.head()

,index,date,open,high,low,close,volume,openint
0,0,1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0
1,1,1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0
2,2,1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0
3,3,1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0
4,4,1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0


In [11]:
def trend_momentum(rets, lookback, lag):
    signal = rets.rolling(lookback, min_periods=5).sum()
    return signal.shift(lag)

In [12]:
rets = (aapl_100[['open']].applymap(float).pct_change()+1).dropna()

In [13]:
rets['date'] = pd.merge(aapl_100[['index', 'date']], 
                        rets, 
                        left_on='index', 
                        right_index=True)['date']

In [14]:
rets.head()

,open,date
1,1.000000,1984-09-10
2,1.003020,1984-09-11
3,1.009079,1984-09-12
4,1.023892,1984-09-13
5,1.002846,1984-09-14


In [15]:
rets.index = rets.date

In [16]:
momentum = trend_momentum(rets, 5,5)

In [17]:
momentum[:10]

,open,date
date,,
1984-09-10,NaN,NaN
1984-09-11,NaN,NaN
1984-09-12,NaN,NaN
1984-09-13,NaN,NaN
1984-09-14,NaN,NaN
1984-09-17,NaN,1984-09-10
1984-09-18,NaN,1984-09-11
1984-09-19,NaN,1984-09-12
1984-09-20,NaN,1984-09-13


In [18]:
rets.head()

,open,date
date,,
1984-09-10,1.000000,1984-09-10
1984-09-11,1.003020,1984-09-11
1984-09-12,1.009079,1984-09-12
1984-09-13,1.023892,1984-09-13
1984-09-14,1.002846,1984-09-14


In [19]:
sum(rets.head()['open'])

5.0388359478648255

In [20]:
temp = rets.head()
temp.shift(2)

,open,date
date,,
1984-09-10,NaN,NaN
1984-09-11,NaN,NaN
1984-09-12,1.000000,1984-09-10
1984-09-13,1.003020,1984-09-11
1984-09-14,1.009079,1984-09-12


In [23]:
rets.columns = list(map(lambda x: 'aapl_' + x, rets.columns))

In [24]:
rets.columns

Index(['aapl_open', 'aapl_date'], dtype='object')